#2021繁體中文場景文字辨識比賽 - 單字辨識範例
這個Colab Notebook的目的是給你一個「中文單字辨識」的簡易範例，將會帶著你從資料的生成，到訓練的撰寫。

今天你如果想要創造一個 人工智慧機器，你希望從一張街景照片上截下一個中文單字，我們就假定是拿坡里中的「拿」好了 ![拿坡里](https://drive.google.com/uc?export=view&id=1oaP4lY7EKag8r0zB8sXKZ-B_DnJxt4l4)

你希望餵給你的人工智慧機器 ![拿](https://drive.google.com/uc?export=view&id=1xAixH6gycmYe5B9RHixcX_TzbENHHCxD) 這個中文單字圖片，它能夠辨識出這個字是「拿」，你會怎麼做呢，讓我們從資料集開始～



## 一、 資料集生成

做深度學習最重要的就是Data，你當然可以走到街上去拍數以千張類似「拿」這樣各式各樣的圖片，再自己標記上正確答案，但那樣的話太花費力氣了，而且不是每個字你都在路上找得到。

所以我們這邊要用的是一個單字圖片的生成工具，你可以指定字典、字型、數量、影像轉換，去生成大量資料。
這個工具是 [Single_char_image_generator](https://github.com/rachellin0105/Single_char_image_generator)，更多詳細資訊可以看它的README.md。



In [20]:
%cd /content/
# 把資料生成工具 clone 下來
!git clone https://github.com/rachellin0105/Single_char_image_generator.git
%cd Single_char_image_generator


/content
fatal: destination path 'Single_char_image_generator' already exists and is not an empty directory.
/content/Single_char_image_generator


In [21]:
# Single_char_image_generator/chars.txt 是字典，預設有102字，可以在上面增減字。這邊因為是示範，我們只留前10個字。
!head -n 40 chars.txt > temp.txt
!mv temp.txt chars.txt

In [ ]:

# 安裝它需要的套件
!python -m pip install -r requirements.txt

# 用一行指令執行生成 
!python OCR_image_generator_single_ch.py --num_per_word=500

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/.caches/ceb1594269364fa1c5230afd0053bf61
Load font(./fonts/chinse_jian/2.ttf) supported chars(40) from cache
/content/.caches/a56a73c2d7a54d049026dfa482bfaac9
Load font(./fonts/chinse_jian/simfang.ttf) supported chars(40) from cache
Resume generating from step 2000
Start generating...
Saving images in directory : output
 18% 7/40 [03:42<17:08, 31.16s/it]

## 使用Pytorch 訓練ResNet-18

首先，import我們要使用到的packages和modules進來。

這裡使用的套件都是Google Colab本身就已經安裝好的，無須再多安裝其他套件。

使用 `torch.cuda.is_available()` 確定現在Cuda是否支援 (能否使用GPU運算)。
若沒有將此Colab開啟GPU功能，此函式會回傳False。


In [ ]:
import torch
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('GPU')
else:
  device = torch.device('cpu')
  print('CPU')


自訂一個屬於自己的資料集，繼承pytorch中的Dataset Class。

實作 `__init__`、`__len__`、`__getitem__`三個函式

In [ ]:
class ChineseCharDataset(Dataset):
  def __init__(self, data_file, root_dir, dict_file):
    # data_file:  標註檔的路徑 (標註檔內容: ImagePath, GroundTruth)
    # root_dir: ImagePath所在的資料夾路徑
    # dict_file: 字典的路徑

    # 使用 pandas 將生成的單字labels.txt當作csv匯入進來
    self.char_dataframe = pd.read_csv(data_file, index_col=False, encoding='utf-8', header=None)
    self.root_dir = root_dir
    with open(dict_file, 'r', encoding='utf-8') as f:
      # 將資料集包含的字集匯入進來
      word_list = [line for line in f.read().split('\n') if line.strip() != '']
      self.dictionary = {word_list[i]: i for i in range(0, len(word_list))}

    print(self.char_dataframe)
    print(self.dictionary)

  def __len__(self):
    return len(self.char_dataframe)

  def __getitem__(self, idx):
    
    # 取得第idx張圖片的path，並將圖片打開
    image_path = os.path.join(self.root_dir, self.char_dataframe.iloc[idx, 0])
    image = Image.open(image_path)
    
    # 取得 Ground Truth 並轉換成數字
    char = self.char_dataframe.iloc[idx, 1]
    char_num = self.dictionary[char]

    
    return (transforms.ToTensor()(image), torch.Tensor([char_num]))
    

In [ ]:
%cd /content/

# 宣告好所有要傳入 ChineseCharDataset 的引數
data_file_path = './Single_char_image_generator/output/labels.txt'
root_dir = './Single_char_image_generator/'
dict_file_path = './Single_char_image_generator/chars.txt'

# 模型儲存位置
save_path = './checkpoint.pt'

# 宣告我們自訂的Dataset，把它包到 Dataloader 中以便我們訓練使用
char_dataset = ChineseCharDataset(data_file_path, root_dir, dict_file_path)
char_dataloader = DataLoader(char_dataset, batch_size=64, shuffle=True, num_workers=2)

__開始訓練__

使用 ResNet-18 訓練我們的單字辨識模型

In [ ]:
# --- Training ---

# 我們使用torchvision提供的 ResNet-18 當作我們的AI模型。 
net = models.resnet18(num_classes=40) # num_classes 為類別數量(幾種不一樣的字)
net = net.to(device) # 傳入GPU
net.train()

optimizer = optim.Adam(net.parameters(), lr=0.005)

# 訓練總共Epochs數
epochs = 30


each_loss = []
for i in tqdm(range(1, epochs + 1)):
  losses = 0
  for idx, data in enumerate(char_dataloader):
    image, label = data
    image = image.to(device)
    label = label.squeeze() # 將不同batch壓到同一個dimension
    label = label.to(device, dtype=torch.long)
    
    net.zero_grad()
    result = net(image)

    # 計算損失函數
    loss = F.cross_entropy(result, label)
    losses += loss.item()
    if idx % 10 == 0:  # 每10個batch輸出一次
      print(f'epoch {i}- loss: {loss.item()}')

    # 計算梯度，更新模型參數
    loss.backward()
    optimizer.step()

  avgloss = losses / len(char_dataloader)
  each_loss.append(avgloss)
  print(f'{i}th epoch end. Avg loss: {avgloss}')

# 儲存模型
torch.save({
  'epoch': epochs,
  'model_state_dict': net.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),  
}, save_path)

# 畫出訓練過程圖表 (Y_axis - loss / X_axis - epoch)
plt.plot(each_loss, '-b', label='loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

    

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms

# 定义计算准确率函数
def compute_accuracy(model, test_dataset):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_dataset:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# 计算准确率
accuracy = compute_accuracy(net, char_dataloader)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# 创建一个与原始模型相同结构的模型实例
net = models.resnet18(num_classes=40)

# 加载保存的权重
checkpoint = torch.load(save_path)
net.load_state_dict(checkpoint['model_state_dict'])

# 设置模型为评估模式
net.eval()

# 准备待推理的图像
image_path = './Single_char_image_generator/output/img_0000014.jpg'
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = transform(image).unsqueeze(0)

# 使用模型进行推理
with torch.no_grad():
    outputs = net(input_image)

# 获取预测结果
_, predicted = torch.max(outputs, 1)
prediction = predicted.item()

# 输出预测结果
print('Prediction:', prediction)


In [ ]:
# 定义类别标签映射
class_labels = ['肉','古','幼','酥','成','傢','婦','汎','貨','理','男','大','老','樹','民','鴻','禾','髮','酒','麗','鹽','容','由','寵','中','速','食','汽','子','院','批','洗','素','我','快','雞','出','動','品','活']  # 替换为你的实际类别标签

# 获取预测结果
_, predicted = torch.max(outputs, 1)
prediction = predicted.item()

# 根据类别索引获取类别标签
predicted_label = class_labels[prediction-1]

image = Image.open(image_path)

plt.imshow(image)
plt.axis('off')
plt.show()
# 输出预测结果
print('Prediction:', predicted_label)
